© 2024 Rebecca J. Rousseau and Justin B. Kinney, *Algebraic and diagrammatic methods for the rule-based modeling of multi-particle complexes*. This work is licensed under a [Creative Commons Attribution License CC-BY 4.0](https://creativecommons.org/licenses/by/4.0/). All code contained herein is licensed under an [MIT license](https://opensource.org/licenses/MIT).
___

# Deterministic simulation for homodimer system

In [1]:
import numpy as np
from itertools import combinations
from scipy import sparse, linalg

In [2]:
# Define "dictionary" of mathematical operators tracking field formation (hat), degradation (check), presence (bar),
# and absence (tilde), as well as identity (id) and "zero" (zero) operators.

mat_dict = dict(
    hat=sparse.csr_matrix(np.array([[0, 1], [0, 0]])),
    bar=sparse.csr_matrix(np.array([[1, 0], [0, 0]])),
    tilde=sparse.csr_matrix(np.array([[0, 0], [0, 1]])),
    check=sparse.csr_matrix(np.array([[0, 0], [1, 0]])),
    id=sparse.csr_matrix(np.array([[1, 0], [0, 1]])),
    zero=sparse.csr_matrix(np.array([[0, 0], [0, 0]]))
)

In [3]:
def multikron(mat_list):
    """Computes Kronecker product of multiple matrices"""
    n = len(mat_list)
    assert n>0
    out_mat = mat_list[0]
    for mat in mat_list[1:]:
        out_mat = sparse.kron(out_mat, mat, format='csc')
    return out_mat

In [4]:
N = 4 # Total number of possible monomer internal states
K = int(2*N + N*(N-1)/2) # Total number of mode operator fields
r_A_cre = 2 # Rate of monomer creation
r_A_deg = 2 # Rate of monomer degradation
r_I_cre = 0.5 # Rate of bond formation
r_I_deg = 1 # Rate of bond degradation

In [5]:
# A_i: k = 1 .. N (monomer mode operators)
# a_i: k = N+1 .. 2N (monomer binding site mode operators)
# I_ij: k = 2N+1 .. 2N+N(N-1)/2 (bond mode operators)

### Create single-digit index for each bond type:

# Define all possible monomer index bond pairs

ij_list = [(i,j) for i in range(N) for j in range(N) if j > i]

# Assign index bond pair to a single-digit index label

dk_to_ij_dict = dict([(dk, ij) for dk, ij in enumerate(ij_list)])

ij_to_dk_dict = dict(zip(dk_to_ij_dict.values(), dk_to_ij_dict.keys()))

In [6]:
def op(field_name, field_type, field_index):
    if field_name=='A':
        i = field_index
        k = i
    
    elif field_name=='a':
        i = field_index
        k = N+i
        
    elif field_name=='I':
        i,j = field_index
        k = 2*N + ij_to_dk_dict[(i,j)]
        
    return multikron([mat_dict[field_type] if l==k else mat_dict['id'] for l in range(K)])

In [7]:
# Compute transition operator W

W = multikron([mat_dict['zero']]*K)
for i in range(N):
    W += r_A_cre*(op('A','hat',i) - op('A','tilde',i))*op('a','tilde',i) + \
         r_A_deg*(op('A','check',i) - op('A','bar',i))*op('a','tilde',i)
for i in range(N):
    for j in range(N):
        if i < j:
            W += r_I_cre*(op('I','hat',(i,j))*op('a','hat',i)*op('a','hat',j)-\
                          op('I','tilde',(i,j))*op('a','tilde',i)*op('a','tilde',j))*\
                              op('A','bar',i)*op('A','bar',j) + \
                 r_I_deg*(op('I','check',(i,j))*op('a','check',i)*op('a','check',j) -\
                          op('I','bar',(i,j))*op('a','bar',i)*op('a','bar',j))*\
                              op('A','bar',i)*op('A','bar',j)

In [8]:
# Compute A counting matrix

A_bar = multikron([mat_dict['zero']]*K)
for i in range(N):
    A_bar += op('a','tilde',i)*op('A','bar',i)  # count free (unbound) monomers

In [9]:
# Compute I counting matrix

I_bar = multikron([mat_dict['zero']]*K)
for i in range(N):
    for j in range(N):
        if i < j: #if i < j
            I_bar += op('I','bar',(i,j))*op('a','bar',i)*op('a','bar',j)*op('A','bar',i)*op('A','bar',j)

In [10]:
# Construct ground state vector

ground_el = sparse.csc_matrix(np.array([0, 1]))
ground_state = multikron([ground_el]*K)
ground_state = ground_state.toarray().squeeze()

In [11]:
# Construct sum vector

sum_vec = np.ones(2**K).T

In [12]:
# Compute evolving system state function

from scipy.sparse.linalg import expm_multiply
t_stop = 10
num_timepoints = 2001
psi_array = expm_multiply(W,ground_state.T,start=0.0, stop=t_stop, num=num_timepoints)

In [13]:
# Compute the number of monomers ("A" count) and homodimers ("I" bond count) at times t

A_of_t = np.zeros(num_timepoints)
I_of_t = np.zeros(num_timepoints)
for t in range(num_timepoints):
    psi_t = psi_array[t,:]
    A_of_t[t] = sum_vec.dot(A_bar.dot(psi_t))
    I_of_t[t] = sum_vec.dot(I_bar.dot(psi_t))

In [14]:
folderpath = "../simulationdata"
np.savetxt(f"{folderpath}/A_of_t_homodimer_N{N}.csv",A_of_t,delimiter=",")
np.savetxt(f"{folderpath}/I_of_t_homodimer_N{N}.csv",I_of_t,delimiter=",")